# Calculating Climatology and Departures from Time Series Data

Author: [Tom Vo](https://github.com/tomvothecoder/) & [Jiwoo Lee](https://github.com/lee1043/)

Updated: 11/07/24 [xcdat v0.7.3]

Related APIs:

- [xarray.Dataset.temporal.climatology()](../generated/xarray.Dataset.temporal.climatology.rst)
- [xarray.Dataset.temporal.departures()](../generated/xarray.Dataset.temporal.departures.rst)


## Overview

Suppose we're working with an `"ersstv5"` dataset (NOAA’s Extended Reconstructed Sea Surface Temperature monthly averages) with time from 1970-01-01 to 2015-01-01.

1. We want to calculate the annual and seasonal cycle climatologies and departures using this file.
2. We want to resample it to hourly frequency and calculate the daily cycle climatologies and departures.

The data used in this example can be found in the [xarray-data repository](https://github.com/pydata/xarray-data).


### Notebook Kernel Setup

Users can [install their own instance of xcdat](../getting-started-guide/installation.rst) and follow these examples using their own environment (e.g., with VS Code, Jupyter, Spyder, iPython) or [enable xcdat with existing JupyterHub instances](../getting-started-guide/getting-started-hpc-jupyter.rst).

First, create the conda environment:

```bash
conda create -n xcdat_notebook -c conda-forge xcdat xesmf matplotlib ipython ipykernel cartopy nc-time-axis gsw-xarray jupyter pooch
```

Then install the kernel from the `xcdat_notebook` environment using `ipykernel` and name the kernel with the display name (e.g., `xcdat_notebook`):

```bash
python -m ipykernel install --user --name xcdat_notebook --display-name xcdat_notebook
```

Then to select the kernel `xcdat_notebook` in Jupyter to use this kernel.


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import xcdat as xc

## 1. Open Sample Datasets


In [ ]:
ds = xc.tutorial.open_dataset("ersstv5", use_cftime=True)

### File 1: Monthly Frequency


In [ ]:
ds_monthly = ds.copy()

ds_monthly

### 2: Hourly Frequency

We're going to resample the dataset to hourly frequency.


In [ ]:
ds_hourly = ds.sel(time=slice("2010-01-01", "2015-01-01"))
ds_hourly = ds_hourly.resample(time="h").mean()
ds_hourly = ds_hourly.bounds.add_bounds(axis="T")

ds_hourly

## 2. Calculate Climatology

Related API: [xarray.Dataset.temporal.climatology()](../generated/xarray.Dataset.temporal.climatology.rst)

In this example, we will be calculating the weighted climatology of the `sst` variable for its seasonal, annual, and daily cycles.

Helpful knowledge:

- Masked (missing) data is automatically handled.
  - The weight of masked (missing) data is excluded when averages are calculated. This is the same as giving them a weight of 0.
- If desired, use the `reference_period` argument to calculate a climatology based on a
  climatological reference period (a subset of the entire time series). If no value is
  provided, the climatological reference period will be the full period covered by the
  dataset.


### Seasonal Climatology

**Groups time coordinates by season**

The `season_config` dictionary keyword argument can be passed to `.climatology()` for more granular configuration. We will be sticking with the default settings.


In [ ]:
season_climo = ds_monthly.temporal.climatology(
    "sst",
    freq="season",
    weighted=True,
    season_config={"dec_mode": "DJF", "drop_incomplete_djf": True},
)

In [ ]:
notnull = pd.notnull(season_climo["sst"][0])
sst_season = season_climo.sst

fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(14, 12))
for i, season in enumerate(("DJF", "MAM", "JJA", "SON")):
    sst_season.isel(time=i).where(notnull).plot.pcolormesh(
        ax=axes[i],
        vmin=-30,
        vmax=30,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="both",
    )
    axes[i].set_ylabel(season)

for ax in axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.axis("tight")
    ax.set_xlabel("")

plt.tight_layout()
fig.suptitle("Seasonal Surface Air Temperature", fontsize=16, y=1.02)

Notice that the time coordinates are `cftime` objects, with each season ("DJF", "MAM", "JJA", and "SON") represented by its middle month.

`cftime` objects are used because the time coordinates are outside the Timestamp-valid range (approximately between years 1678 and 2262).

- More info here: https://xarray.pydata.org/en/v2022.03.0/user-guide/weather-climate.html#non-standard-calendars-and-dates-outside-the-timestamp-valid-range


In [ ]:
season_climo.time

### Custom Seasonal Climatology

**Groups time coordinates by season**

To calculate custom seasonal cycle climatology, we must first define our custom seasons using the `season_config` dictionary and the `"custom_seasons"` key.

`"custom_seasons"` must be a list of sublists containing month strings, with each sublist
representing a custom season.

- Month strings must be in the three letter format (e.g., 'Jan')
- Each month must be included once in a custom season
- Order of the months in each custom season does not matter
- Custom seasons can vary in length


In [ ]:
custom_seasons = [
    ["Jan", "Feb", "Mar"],  # "JanFebMar"
    ["Apr", "May", "Jun"],  # "AprMayJun"
    ["Jul", "Aug", "Sep"],  # "JunJulAug"
    ["Oct", "Nov", "Dec"],  # "OctNovDec"
]

c_season_climo = ds_monthly.temporal.climatology(
    "sst",
    freq="season",
    weighted=True,
    season_config={"custom_seasons": custom_seasons},
)

In [ ]:
c_season_climo.sst

In [ ]:
notnull = pd.notnull(c_season_climo["sst"][0])
sst_c_season = c_season_climo.sst

fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(14, 12))
for i, season in enumerate(sst_c_season.attrs["custom_seasons"]):
    sst_c_season.isel(time=i).where(notnull).plot.pcolormesh(
        ax=axes[i],
        vmin=-30,
        vmax=30,
        cmap="Spectral_r",
        add_colorbar=True,
        extend="both",
    )
    axes[i].set_ylabel(season)

for ax in axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.axis("tight")
    ax.set_xlabel("")

plt.tight_layout()
fig.suptitle("Seasonal Surface Air Temperature", fontsize=16, y=1.02)

### Annual Climatology

**Groups time coordinates by month**


In [ ]:
annual_climo = ds_monthly.temporal.climatology("sst", freq="month", weighted=True)

In [ ]:
annual_climo.sst

### Daily Climatology

**Groups time coordinates by month and day.**

Leap days (if present) are dropped if the CF calendar type is `"gregorian"`, `"proleptic_gregorian"`, or `"standard"`.


In [ ]:
daily_climo = ds_hourly.temporal.climatology("sst", freq="day", weighted=True)

In [ ]:
daily_climo.sst

## 3. Calculate Departures (Anomalies)

Related API: [xarray.Dataset.temporal.departures()](../generated/xarray.Dataset.temporal.departures.rst)

In this example, we will be calculating the weighted departures of the `sst` variable for its seasonal, annual, and daily cycles.

Helpful knowledge:

- What are anomalies?
  - In climatology, “anomalies” refer to the difference between the value during a given time
    interval (e.g., the January average surface air temperature) and the long-term average value for
    that time interval (e.g., the average surface temperature over the last 30 Januaries).
- How is the climatology calculated?
  - In the departures API, the reference climatology is calculated internally so there is no need to pass one to this method.
  - You can still calculate the reference climatology using the climatology API.
  - If desired, use the `reference_period` argument to calculate anomalies relative to a
    climatological reference period (a subset of the entire time series). If no value is
    provided, the climatological reference period will be the full period covered by the
    dataset.
- Masked (missing) data is automatically handled.
  - The weight of masked (missing) data is excluded when averages are calculated. This is the same as giving them a weight of 0.


### Seasonal Anomalies

The `season_config` dictionary keyword argument can be passed to `.departures()` for more granular configuration. We will be sticking with the default settings.


In [ ]:
season_departures = ds_monthly.temporal.departures(
    "sst",
    freq="season",
    weighted=True,
    season_config={"dec_mode": "DJF", "drop_incomplete_djf": True},
)

In [ ]:
season_departures.sst

### Custom Seasonal Anomalies

To calculate custom seasonal cycle anomalies, we must first define our custom seasons using the `season_config` dictionary and the `"custom_seasons"` key.

`"custom_seasons"` must be a list of sublists containing month strings, with each sublist
representing a custom season.

- Month strings must be in the three letter format (e.g., 'Jan')
- Each month must be included once in a custom season
- Order of the months in each custom season does not matter
- Custom seasons can vary in length


In [ ]:
custom_seasons = [
    ["Jan", "Feb", "Mar"],  # "JanFebMar"
    ["Apr", "May", "Jun"],  # "AprMayJun"
    ["Jul", "Aug", "Sep"],  # "JulAugSep"
    ["Oct", "Nov", "Dec"],  # "OctNovDec"
]
c_season_departs = ds_monthly.temporal.departures(
    "sst",
    freq="season",
    weighted=True,
    season_config={"custom_seasons": custom_seasons},
)

In [ ]:
c_season_departs.sst

### Annual Anomalies


In [ ]:
annual_departures = ds_monthly.temporal.departures("sst", freq="month", weighted=True)

In [ ]:
annual_departures.sst

### Daily Anomalies

Leap days (if present) are dropped if the CF calendar type is `"gregorian"`, `"proleptic_gregorian"`, or `"standard"`.


In [ ]:
daily_departures = ds_hourly.temporal.departures("sst", freq="day", weighted=True)

In [ ]:
daily_departures.sst